# Ejemplo Aprendizage por refuerzo

El siguiente ejemplo quiere mostrar un caso sencillo de aprendizage por refuerzo utilizando Python y la libreria de OPENAI Gym.

El ejemplo trata de un taxi que necesita desplazar personas entre unos puntos (RGBY). El taxi puede realizar acciones como lo son desplazarse arriba, abajo, derecha, izquierda, recoger y dejar al pasajero. 

Cada movimiento resta un punto al taxi, por cada vez que se recoje o se deja a alguien en un sitio incorrecto se restan 10 puntos y por cada vez que un pasajero es dejado en su destino se suman 20 puntos.

El siguiente codigo ha sido tomado de https://builtin.com/data-science/reinforcement-learning-python y https://gist.github.com/enzodeal/ae196380f80df9ff522d34f84f324e51. 

El codigo anterior busca simular de manera aleatoria el comportamiento del taxi y termina cuando el taxi deja por primera vez a un pasajero.

In [1]:
import gym #una de las bibliotecas más utilizadas para resolver problemas de aprendizaje por refuerzo
from IPython.display import clear_output
from time import sleep

# Creating thr env
#Todos los modelos y la interfaz para este problema ya están configurados en Gym y nombrados bajo  Taxi-V3
#Env es el núcleo de OpenAi Gym, que es la interfaz de entorno unificado. Los siguientes son los métodos env que nos serán muy útiles:

  #env.reset : restablece el entorno y devuelve un estado inicial aleatorio.
  #env.step (acción):  Paso del entorno por un paso de tiempo.
  #env.step (acción): Devuelve las siguientes variables:
    #observation: Observaciones del medio ambiente.
    #reward: Si su acción fue beneficiosa o no.
    #done: Indica si hemos recogido y dejado con éxito a un pasajero, también llamado un episodio.
    #info: Información adicional como el rendimiento y la latencia con fines de depuración.
    #env.render: Renderiza un cuadro del entorno (útil para visualizar el entorno).
    

#Hay una recompensa de -1 por cada acción y una recompensa adicional de +20 por entregar al pasajero. Hay una recompensa de -10 por ejecutar acciones 'recoger' y 'dejar' ilegalmente.

env = gym.make("Taxi-v3").env

print(env)
env.s = 328 #


# Setting the number of iterations, penalties and reward to zero,
epochs = 0
penalties, reward = 0, 0

frames = []

done = False

while not done: #Se crea bucle infinito y termina cuando deja la persona que recogio
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1

    # Put each rendered frame into the dictionary for animation
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'action': action,
        'reward': reward
    }
    )

    epochs += 1

# Printing all the possible actions, states, rewards.
def show(x):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

        
show(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)

Timestep: 334
State: 108
Action: 1
Reward: -1


KeyboardInterrupt: ignored

El siguiente codigo intenta aplicar un algoritmo de Qlearn para optimizar la manera en que el taxi se encarga de recojer y dejar en sus destinos a los ususarios.

In [2]:
import numpy as np
import random


# Init Taxi-V2 Env
env = gym.make("Taxi-v3").env

# Init arbitary values
q_table = np.zeros([env.observation_space.n, env.action_space.n])

# Hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1


all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    # Init Vars
    epochs, penalties, reward, = 0, 0, 0
    done = False

    while not done:
        if random.uniform(0, 1) < epsilon:
            # Check the action space
            action = env.action_space.sample()
        else:
            # Check the learned values
            action = np.argmax(q_table[state])

        next_state, reward, done, info = env.step(action)

        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])

        # Update the new value
        new_value = (1 - alpha) * old_value + alpha * \
            (reward + gamma * next_max)
        q_table[state, action] = new_value

        if reward == -10:
            penalties += 1

        state = next_state
        epochs += 1

    if i % 100 == 0:
        clear_output(wait=True)
        print("Episode: {i}")

print("Training finished.")

Episode: {i}
Training finished.


La siguiente matriz almacena los resultados del algoritmo anterior filas estados columnas acciones

In [3]:
import pandas as pd

results = pd.DataFrame(q_table)
results



,0,1,2,3,4,5
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-2.418371,-2.363951,-2.418371,-2.363951,-2.273252,-11.363951
2,-1.870144,-1.450240,-1.870144,-1.450240,-0.750400,-10.450240
3,-2.363951,-2.273252,-2.363951,-2.273252,-2.122086,-11.273252
4,-2.496192,-2.496768,-2.496192,-2.496801,-9.678722,-9.492347
...,...,...,...,...,...,...
495,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
496,-2.143547,-2.122033,-2.145673,-2.122033,-6.288920,-7.639420
497,-1.041943,0.416000,-1.101089,-1.230037,-4.128253,-4.131789
498,-2.181177,-2.122005,-2.158237,-2.122011,-5.734670,-4.441863


El siguiente codigo ejecuta el modelo que creamos anteriormente con la matriz q para demostrar como trabaja el modelo.

In [4]:
#probando algoritmo
total_epoch, total_penalties = 0, 0
episodes = 5

for _ in range(episodes):
    state = env.reset()
    epochs, penalties, reward = 0, 0, 0
    
    finished = False
    
    while not finished:
        action = np.argmax(q_table[state])
        state, reward, finished, info = env.step(action)

        if reward == -10:
            penalties += 1

        epochs += 1
        
        clear_output(wait=True)
        env.render()
        sleep(.25)

    total_penalties += penalties
    total_epoch += epochs

print(f"Results after {episodes} episodes:")
print(f"Mean steps: {total_epoch / episodes}")
print(f"Mean penalties: {total_penalties / episodes}")

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)
Results after 5 episodes:
Mean steps: 11.8
Mean penalties: 0.0
